In [5]:
import cProfile
import av
from tqdm import tqdm
import cv2
import numpy as np
import os



def process_video_frame_by_frame(args, skip_seconds=11):
    """逐幀處理影片，從第 skip_seconds 秒開始"""
    video_path, transform_matrix, output_dir, duration, fps_output, resize_dim = args

    try:
        container = av.open(video_path)
    except av.AVError as e:
        raise ValueError(f"無法打開影片: {video_path}，錯誤: {e}")

    stream = container.streams.video[0]

    original_fps = stream.average_rate if stream.average_rate is not None else 1 / stream.time_base
    frame_step = max(1, int(original_fps // fps_output))+1  # 計算跳幀數
    total_frames = int(stream.frames if stream.frames else stream.duration * original_fps)
    max_frames = total_frames if duration is None else min(total_frames, int(duration * original_fps))

    # 計算跳過的起始幀
    start_frame = int(skip_seconds * original_fps)

    # 跳過指定的幀數
    container.seek(int(start_frame / original_fps / stream.time_base))

    # 準備輸出文件
    os.makedirs(output_dir, exist_ok=True)
    base_name, ext = os.path.splitext(os.path.basename(video_path))
    out_path = os.path.join(output_dir, f"{base_name}_bottom{ext}")
    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps_output, resize_dim)

    print(f"開始處理影片: {video_path}, 跳過前 {skip_seconds} 秒, 總幀數: {max_frames}, 原始 FPS: {original_fps}, 跳幀數: {frame_step}")

    with tqdm(total=max_frames // frame_step, desc=f"Processing {os.path.basename(video_path)}") as pbar:
        for i, frame in enumerate(container.decode(video=0)):
            if i < start_frame:
                continue  # 跳過起始幀
            if i >= max_frames + start_frame:
                break
            if (i - start_frame) % frame_step == 0:  # 根據跳幀數選擇幀
                # 轉換幀為 OpenCV 格式
                img = frame.to_ndarray(format="bgr24")
                warped_frame = cv2.warpPerspective(img, transform_matrix, (resize_dim[0], resize_dim[1]))  # 確保輸出尺寸匹配
                warped_frame = warped_frame.astype(np.uint8)  # 確保數據類型正確
                out.write(warped_frame)  # 寫入處理後的幀

                pbar.update(1)

    out.release()
    print(f"處理完成，輸出文件：{out_path}")



def profile_process_video(args):
    """對 process_video_frame_by_frame 函數進行性能分析"""
    profiler = cProfile.Profile()
    try:
        profiler.enable()
        process_video_frame_by_frame(args)
    finally:
        profiler.disable()
        base_name = os.path.splitext(os.path.basename(args[0]))[0]
        profiler.dump_stats(f"profile.prof")
        print(f"性能分析已保存為: profile.prof")


def select_roi_with_av(video_path, resize_dim):
    """使用 PyAV 讓用戶選擇 ROI 並返回透視變換矩陣"""
    try:
        container = av.open(video_path)
    except av.AVError as e:
        raise ValueError(f"無法打開影片: {video_path}，錯誤: {e}")

    stream = container.streams.video[0]

    # 獲取第一幀作為預覽
    for frame in container.decode(video=0):
        img = frame.to_ndarray(format="bgr24")
        break

    if img is None:
        raise ValueError(f"無法從影片 {video_path} 中讀取首幀")

    cv2.namedWindow("Select ROI")
    points = []

    def mouse_callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            print(f"點選點: {x, y}")
            if len(points) == 4:
                cv2.setMouseCallback("Select ROI", lambda *args: None)

    cv2.setMouseCallback("Select ROI", mouse_callback)

    while len(points) < 4:
        preview_frame = img.copy()
        for point in points:
            cv2.circle(preview_frame, point, 5, (0, 0, 255), -1)
        cv2.imshow("Select ROI", preview_frame)
        key = cv2.waitKey(1)
        if key & 0xFF == 27:  # 按下 ESC 鍵取消
            print("用戶中止操作")
            cv2.destroyAllWindows()
            return None

    cv2.destroyAllWindows()

    # 計算透視變換矩陣
    dst_points = np.float32([[0, 0], [resize_dim[1], 0], [resize_dim[1], resize_dim[0]], [0, resize_dim[0]]])
    src_points = np.float32(points)
    transform_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    print("透視變換矩陣:\n", transform_matrix)

    return transform_matrix



def select_files():
    """選擇多部影片和輸出資料夾"""
    from tkinter import Tk, filedialog
    root = Tk()
    root.attributes('-topmost', True)
    root.withdraw()

    video_paths = filedialog.askopenfilenames(title="選擇多部影片")
    output_dir = filedialog.askdirectory(title="選擇輸出資料夾")
    if not video_paths or not output_dir:
        print("未選擇影片或輸出資料夾")
        return None, None

    return video_paths, output_dir


def process_multiple_videos(video_paths, output_dir, duration=1 * 60, fps_output=30, resize_dim=(480, 480)):
    """批量處理多部影片"""
    roi_matrices = []

    for video_path in video_paths:
        print(f"正在選擇影片 ROI: {os.path.basename(video_path)}")
        roi_matrix = select_roi_with_av(video_path, resize_dim)
        if roi_matrix is None:
            print(f"跳過影片: {os.path.basename(video_path)}")
            continue
        roi_matrices.append((video_path, roi_matrix))

    args = [(video_path, matrix, output_dir, duration, fps_output, resize_dim) for video_path, matrix in roi_matrices]

    if args:
        profile_process_video(args[0])
        for arg in args[1:]:  # 順序處理其餘影片
            process_video_frame_by_frame(arg)
    print("所有影片處理完成！")


if __name__ == "__main__":
    try:
        video_files, output_folder = select_files()
        if video_files and output_folder:
            process_multiple_videos(video_files, output_folder)
    except Exception as e:
        print(f"出現錯誤: {e}")


正在選擇影片 ROI: 6L_5_bottom.MP4
點選點: (0, 1)
點選點: (477, 1)
點選點: (477, 476)
點選點: (1, 477)
透視變換矩陣:
 [[ 1.00416641e+00 -2.10959330e-03  2.10959330e-03]
 [ 0.00000000e+00  1.00628489e+00 -1.00628489e+00]
 [-4.41349124e-06 -4.39498604e-06  1.00000000e+00]]
開始處理影片: D:/works/Data_analysis/projects/grid_walking/2024_09/dlc/gridwalking_202409-MX-2024-11-19/videos/6L_5_bottom.MP4, 跳過前 11 秒, 總幀數: 1800, 原始 FPS: 30, 跳幀數: 2


Processing 6L_5_bottom.MP4: 100%|██████████| 900/900 [00:03<00:00, 236.98it/s]

處理完成，輸出文件：D:/works/Data_analysis/projects/grid_walking/2024_09/dlc/gridwalking_202409-MX-2024-11-19/videos/test\6L_5_bottom_bottom.MP4
性能分析已保存為: profile.prof
所有影片處理完成！


In [3]:
import pstats
stats = pstats.Stats("profile.prof")
stats.strip_dirs().sort_stats("time").print_stats(10)

Tue Nov 19 15:03:53 2024    profile.prof

         277168 function calls (277156 primitive calls) in 3.745 seconds

   Ordered by: internal time
   List reduced from 210 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    2.741    2.741    3.745    3.745 1147145791.py:10(process_video_frame_by_frame)
      900    0.525    0.001    0.525    0.001 {warpPerspective}
     1804    0.200    0.000    0.200    0.000 {built-in method nt.stat}
      900    0.047    0.000    0.047    0.000 {method 'astype' of 'numpy.ndarray' objects}
     9010    0.040    0.000    0.060    0.000 <frozen importlib._bootstrap_external>:96(_path_join)
     1802    0.017    0.000    0.329    0.000 <frozen importlib._bootstrap>:921(_find_spec)
      288    0.017    0.000    0.017    0.000 {method 'acquire' of '_thread.lock' objects}
     1802    0.017    0.000    0.282    0.000 <frozen importlib._bootstrap_external>:1536(find_spec)
1806/1802    0.011    0